# RAG with Vertex AI Vector Search 2.0 (Agent Engine)

#### Overview

This notebook demonstrates how to interact with a RAG (Retrieval-Augmented Generation) agent that has been deployed to **Vertex AI Agent Engine**. It focuses on using the **Vertex AI Agent Development Kit (ADK)** capabilities to manage sessions and perform real-time streaming queries.

#### Key Features
*   **Environment Configuration**: Quick setup for Project ID, Location, and Agent Engine identifiers.
*   **Agent Engine Integration**: Connecting to a live `ReasoningEngine` resource deployed on Google Cloud.
*   **Stateful Sessions**: Managing persistent conversation sessions to maintain context across multiple user turns.
*   **Async Streaming**: Implementing asynchronous streaming queries for a responsive, "typewriter-style" user experience.
*   **Resource Cleanup**: Guidance on managing and deleting reasoning engines to optimize cloud costs.

#### Prerequisites
Before proceeding with this notebook, ensure you have:
1.  Completed the data ingestion process into Vertex AI Vector Search 2.0.
2.  Deployed your agent using the `adk deploy agent_engine` command.
3.  Recorded the **Agent Engine ID** from your deployment summary.

## Configure Your Google Cloud Project

Now, let's configure the SDK for your Google Cloud environment. You will need the following information:

- Project ID: Found in the Google Cloud Console.
- Project Location: The region where your resources are deployed (e.g., us-central1).
- Agent Engine ID: This ID is generated after successfully deploying your agent. You can find it in the terminal output after running `adk deploy agent_engine`.
  - Example: `Successfully created remote agent: .../reasoningEngines/<AGENT_ENGINE_ID>`

Please replace the placeholder values below with your actual Project ID, Location, and Agent Engine ID.

In [ ]:
import os

# TODO: Replace with your project ID, location, and agent engine ID

PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT") # @param {type: "string"}
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION") # @param {type: "string"}
AGENT_ENGINE_ID = "[your-agent-engine-id]" # @param {type: "string"}

PROJECT_ID, LOCATION, AGENT_ENGINE_ID

## Connect to Your Deployed Agent

Let's connect to it so we can start asking questions.

**You need to do one thing:** Copy the resource name from the deployment output above (it looks like `projects/123.../locations/us-central1/reasoningEngines/456...`) and paste it into the `agent_resource_name` variable below.

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines

# Initialize the client with API key
client = vertexai.Client(project=PROJECT_ID, location=LOCATION)

print("✅ Google Cloud environment configured")
print(f"   Project: {PROJECT_ID}")
print(f"   Location: {LOCATION}")

In [ ]:
agent_resource_name = f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{AGENT_ENGINE_ID}"

# Get the deployed agent
remote_agent = client.agent_engines.get(name=agent_resource_name)

print("✅ Agent retrieved successfully!")
print(f"  Resource: {remote_agent.api_resource.name}")

## Test the Deployed AdkApp

Let's make sure everything works! We'll ask the agent about Google ADK - a topic that requires current information from the web.

Let's query the agent using the session we just created. Notice how we pass the session_id - this enables the agent to maintain context across multiple turns.

**Create a Session**

Before querying the deployed agent, we need to create a session. Sessions maintain conversation history and state across multiple queries.

In [ ]:
# Create a session for this user
remote_session = await remote_agent.async_create_session(user_id="demo_user_adk")

print("✅ Session created!")
print(f"   Session ID: {remote_session['id']}")
print(f"   User ID: {remote_session['userId']}")
print(f"   App Name: {remote_session['appName']}")

In [ ]:
# Query the agent with streaming responses
user_query = 'What is ADK?'
print(f"🔍 Query: '{user_query}'")
print("-" * 70)

async for item in remote_agent.async_stream_query(
    user_id="demo_user_adk",
    session_id=remote_session["id"],
    message=user_query,
):
    # Print the response content
    if "content" in item and item["content"] and "parts" in item["content"]:
        for part in item["content"]["parts"]:
            if "text" in part:
                print(part["text"], end="", flush=True)

print("\n" + "-" * 70)
print("✅ Query completed successfully!")

## Cleaning up

When you're done experimenting, it's a good idea to delete your deployed agents to avoid unexpected charges. 